## 1️⃣ Setup & Check GPU

First, check if PyTorch can detect your GPU (AMD or NVIDIA).

In [1]:
# Check PyTorch and GPU availability
import torch
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available (NVIDIA): {torch.cuda.is_available()}")

# Check for DirectML (AMD GPU)
try:
    import torch_directml
    dml_device = torch_directml.device()
    print(f"✅ DirectML Available (AMD GPU): Yes")
    print(f"   Device: {dml_device}")
    device = dml_device
except ImportError:
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"✅ Using NVIDIA GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device('cpu')
        print(f"⚠️ No GPU detected - using CPU")
        print(f"\n💡 To enable AMD GPU support, install: pip install torch-directml")

print(f"\n🎯 Training device: {device}")

PyTorch: 2.9.1+cpu
CUDA Available (NVIDIA): False
⚠️ No GPU detected - using CPU

💡 To enable AMD GPU support, install: pip install torch-directml

🎯 Training device: cpu


### 🎮 AMD GPU Setup (If Not Detected)

If DirectML is not available, install it:

```bash
pip install torch-directml
```

Then restart the kernel and run the cell above again.

## 2️⃣ Configuration

**IMPORTANT:** Update `DATASET_ROOT` to point to your local dataset folder.

In [2]:
import os
import json
import numpy as np
import random
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
from PIL import Image
import gc

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split

# ========================================
# 📁 UPDATE THIS PATH TO YOUR LOCAL DATASET
# ========================================
DATASET_ROOT = Path('E:/FasalVaidya/Leaf Nutrient Data Sets')
MODEL_OUTPUT = Path('E:/FasalVaidya/backend/ml/models_pytorch')

# Create output directory
MODEL_OUTPUT.mkdir(parents=True, exist_ok=True)

# Verify dataset exists
if DATASET_ROOT.exists():
    print(f"✅ Dataset found at: {DATASET_ROOT}")
    print(f"📂 Contents: {[f.name for f in DATASET_ROOT.iterdir() if f.is_dir()][:5]}...")
else:
    print(f"❌ Dataset NOT found at: {DATASET_ROOT}")
    print("Please update DATASET_ROOT to your local path")

✅ Dataset found at: E:\FasalVaidya\Leaf Nutrient Data Sets
📂 Contents: ['Ashgourd Nutrients', 'Banana leaves Nutrient', 'Bittergourd Nutrients', 'Coffee Nutrients', 'Cucumber Nutrients']...


## 3️⃣ Crop Configurations

Same crop configurations as TensorFlow version.

In [3]:
# Seed for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)

# Image settings
IMG_SIZE = 224
MAX_SAMPLES_PER_CLASS = 2000

# Crop configurations (same as TensorFlow version)
CROP_CONFIGS = {
    'rice': {
        'name': 'Rice',
        'dataset_path': DATASET_ROOT / 'Rice Nutrients',
        'class_mapping': {
            'Nitrogen(N)': {'N': 1, 'P': 0, 'K': 0, 'Mg': 0},
            'Phosphorus(P)': {'N': 0, 'P': 1, 'K': 0, 'Mg': 0},
            'Potassium(K)': {'N': 0, 'P': 0, 'K': 1, 'Mg': 0},
        },
        'has_healthy': False,
        'outputs': ['N', 'P', 'K', 'Mg'],
    },
    'tomato': {
        'name': 'Tomato',
        'dataset_path': DATASET_ROOT / 'Tomato Nutrients',
        'use_train_folder': True,
        'class_mapping': {
            'Tomato - Healthy': {'N': 0, 'P': 0, 'K': 0, 'Mg': 0},
            'Tomato - Nitrogen Deficiency': {'N': 1, 'P': 0, 'K': 0, 'Mg': 0},
            'Tomato - Potassium Deficiency': {'N': 0, 'P': 0, 'K': 1, 'Mg': 0},
            'Tomato - Nitrogen and Potassium Deficiency': {'N': 1, 'P': 0, 'K': 1, 'Mg': 0},
            'Tomato - Jassid and Mite': {'N': 0, 'P': 0, 'K': 0, 'Mg': 0},
            'Tomato - Leaf Miner': {'N': 0, 'P': 0, 'K': 0, 'Mg': 0},
            'Tomato - Mite': {'N': 0, 'P': 0, 'K': 0, 'Mg': 0},
        },
        'has_healthy': True,
        'outputs': ['N', 'P', 'K', 'Mg'],
    },
    'wheat': {
        'name': 'Wheat',
        'dataset_path': DATASET_ROOT / 'Wheat Nitrogen',
        'use_train_folder': True,
        'class_mapping': {
            'control': {'N': 0, 'P': 0, 'K': 0, 'Mg': 0},
            'deficiency': {'N': 1, 'P': 0, 'K': 0, 'Mg': 0},
        },
        'has_healthy': True,
        'outputs': ['N', 'P', 'K', 'Mg'],
    },
    'maize': {
        'name': 'Maize',
        'dataset_path': DATASET_ROOT / 'Maize Nutrients',
        'use_train_folder': True,
        'class_mapping': {
            'ALL Present': {'N': 0, 'P': 0, 'K': 0, 'Mg': 0},
            'NAB': {'N': 1, 'P': 0, 'K': 0, 'Mg': 0},
            'PAB': {'N': 0, 'P': 1, 'K': 0, 'Mg': 0},
            'KAB': {'N': 0, 'P': 0, 'K': 1, 'Mg': 0},
            'ALLAB': {'N': 1, 'P': 1, 'K': 1, 'Mg': 0},
            'ZNAB': {'N': 0, 'P': 0, 'K': 0, 'Mg': 0},
        },
        'has_healthy': True,
        'outputs': ['N', 'P', 'K', 'Mg'],
    },
}

print(f"📋 Available crops: {list(CROP_CONFIGS.keys())}")

📋 Available crops: ['rice', 'tomato', 'wheat', 'maize']


## 4️⃣ PyTorch Dataset & DataLoader

In [4]:
class NutrientDataset(Dataset):
    """PyTorch Dataset for nutrient deficiency detection."""
    
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        try:
            img = Image.open(img_path).convert('RGB')
            if self.transform:
                img = self.transform(img)
            return img, torch.FloatTensor(label)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            # Return a dummy image
            return torch.zeros(3, IMG_SIZE, IMG_SIZE), torch.FloatTensor(label)


def get_image_paths_and_labels(crop_id):
    """Get list of image paths and their labels."""
    config = CROP_CONFIGS[crop_id]
    dataset_path = config['dataset_path']
    class_mapping = config['class_mapping']
    use_train_folder = config.get('use_train_folder', False)
    
    if use_train_folder:
        dataset_path = dataset_path / 'train'
    
    if not dataset_path.exists():
        raise FileNotFoundError(f"Dataset not found: {dataset_path}")
    
    image_paths = []
    labels = []
    class_counts = {}
    
    print(f"\n📂 Scanning {config['name']} from: {dataset_path}")
    
    for folder_name, label_dict in class_mapping.items():
        folder_path = dataset_path / folder_name
        if not folder_path.exists():
            print(f"  ⚠️ Folder not found: {folder_name}")
            continue
        
        # Get image files
        img_files = list(folder_path.glob('*.jpg')) + list(folder_path.glob('*.jpeg')) + \
                    list(folder_path.glob('*.png')) + list(folder_path.glob('*.JPG')) + \
                    list(folder_path.glob('*.JPEG')) + list(folder_path.glob('*.PNG'))
        
        # Limit samples per class
        if len(img_files) > MAX_SAMPLES_PER_CLASS:
            img_files = random.sample(img_files, MAX_SAMPLES_PER_CLASS)
        
        class_counts[folder_name] = len(img_files)
        
        for img_path in img_files:
            image_paths.append(str(img_path))
            label = [label_dict.get('N', 0), label_dict.get('P', 0), 
                     label_dict.get('K', 0), label_dict.get('Mg', 0)]
            labels.append(label)
    
    print(f"  📊 Class distribution: {class_counts}")
    print(f"  ✅ Found {len(image_paths)} images")
    
    return image_paths, np.array(labels, dtype=np.float32)


# Data transforms
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("✅ PyTorch Dataset and transforms ready")

✅ PyTorch Dataset and transforms ready


## 5️⃣ Model Architecture

In [5]:
class NutrientDeficiencyModel(nn.Module):
    """PyTorch model for nutrient deficiency detection."""
    
    def __init__(self, backbone='efficientnet_b0', num_outputs=4, pretrained=True):
        super().__init__()
        
        # Load pretrained backbone
        if backbone == 'efficientnet_b0':
            self.backbone = models.efficientnet_b0(weights='DEFAULT' if pretrained else None)
            num_features = self.backbone.classifier[1].in_features
            self.backbone.classifier = nn.Identity()  # Remove original classifier
        elif backbone == 'resnet50':
            self.backbone = models.resnet50(weights='DEFAULT' if pretrained else None)
            num_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
        elif backbone == 'mobilenet_v3_large':
            self.backbone = models.mobilenet_v3_large(weights='DEFAULT' if pretrained else None)
            num_features = self.backbone.classifier[0].in_features
            self.backbone.classifier = nn.Identity()
        else:
            raise ValueError(f"Unknown backbone: {backbone}")
        
        # Custom classification head
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.Dropout(0.3),
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_outputs),
            nn.Sigmoid()  # For multi-label classification
        )
    
    def forward(self, x):
        features = self.backbone(x)
        return self.classifier(features)


def create_model(backbone='efficientnet_b0', num_outputs=4, device='cpu'):
    """Create and initialize model."""
    model = NutrientDeficiencyModel(backbone, num_outputs, pretrained=True)
    model = model.to(device)
    return model


# Test model creation
test_model = create_model(device=device)
total_params = sum(p.numel() for p in test_model.parameters())
print(f"✅ Model created: {total_params:,} parameters")
print(f"   Device: {device}")

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\asus/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:03<00:00, 6.46MB/s]

✅ Model created: 4,371,968 parameters
   Device: cpu


## 6️⃣ Training Function

In [ ]:
def train_crop_model(crop_id, epochs=50, batch_size=16, backbone='efficientnet_b0', lr=1e-3):
    """
    Train a model for a specific crop using PyTorch.
    
    Args:
        crop_id: Crop identifier (e.g., 'rice', 'tomato')
        epochs: Number of training epochs
        batch_size: Batch size
        backbone: Model backbone
        lr: Learning rate
    """
    config = CROP_CONFIGS[crop_id]
    print(f"\n{'='*60}")
    print(f"🌱 Training model for: {config['name']}")
    print(f"{'='*60}")
    
    # Create output directory
    output_dir = MODEL_OUTPUT / crop_id
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Get image paths and labels
    image_paths, labels = get_image_paths_and_labels(crop_id)
    
    if len(image_paths) == 0:
        print(f"❌ No images found for {crop_id}")
        return None, None
    
    # Split into train/val
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        image_paths, labels, test_size=0.2, random_state=SEED
    )
    print(f"\n📊 Train: {len(train_paths)}, Validation: {len(val_paths)}")
    
    # Create datasets and dataloaders
    train_dataset = NutrientDataset(train_paths, train_labels, train_transform)
    val_dataset = NutrientDataset(val_paths, val_labels, val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    # Create model
    model = create_model(backbone=backbone, device=device)
    
    # Loss and optimizer
    criterion = nn.BCELoss()  # Binary cross-entropy for multi-label
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4, verbose=True)
    
    # Training loop
    best_val_loss = float('inf')
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    
    print(f"\n🟢 Training for {epochs} epochs...")
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        
        for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, targets = images.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, targets in val_loader:
                images, targets = images.to(device), targets.to(device)
                outputs = model(images)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                
                # Calculate accuracy (threshold at 0.5)
                predicted = (outputs > 0.5).float()
                correct += (predicted == targets).all(dim=1).sum().item()
                total += targets.size(0)
        
        val_loss /= len(val_loader)
        val_acc = correct / total
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"  Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), output_dir / f'{crop_id}_best.pth')
            print(f"  💾 Saved best model (val_loss: {val_loss:.4f})")
        
        scheduler.step(val_loss)
    
    # Load best model
    model.load_state_dict(torch.load(output_dir / f'{crop_id}_best.pth'))
    
    # Final evaluation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            
            predicted = (outputs > 0.5).float()
            correct += (predicted == targets).all(dim=1).sum().item()
            total += targets.size(0)
    
    val_loss /= len(val_loader)
    val_acc = correct / total
    
    print(f"\n📈 Final Evaluation:")
    print(f"  Val Loss: {val_loss:.4f}")
    print(f"  Val Accuracy: {val_acc:.4f}")
    
    # Save metadata
    metadata = {
        'crop_id': crop_id,
        'crop_name': config['name'],
        'backbone': backbone,
        'outputs': ['N', 'P', 'K', 'Mg'],
        'val_accuracy': float(val_acc),
        'val_loss': float(val_loss),
        'trained_at': datetime.now().isoformat(),
        'train_samples': len(train_paths),
        'val_samples': len(val_paths),
        'device': str(device),
        'framework': 'pytorch',
    }
    
    with open(output_dir / 'metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\n💾 Model saved to: {output_dir}")
    
    # Clear memory
    del model, train_loader, val_loader
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return history, val_acc

print("✅ Training function ready")

✅ Training function ready


: 

## 7️⃣ Train a Single Crop (Demo)

Run this cell to train a model for a single crop.

In [ ]:
# ========================================
# 🎯 SELECT CROP TO TRAIN
# ========================================
CROP_TO_TRAIN = 'rice'  # Change this: rice, tomato, wheat, maize
EPOCHS = 30             # Increase for better results (50-100 recommended)
BATCH_SIZE = 16         # Adjust based on your GPU memory (16-32)
BACKBONE = 'efficientnet_b0'  # or 'resnet50', 'mobilenet_v3_large'

# Verify crop exists
if CROP_TO_TRAIN in CROP_CONFIGS:
    print(f"🚀 Starting training for: {CROP_TO_TRAIN}")
    print(f"🎯 Using device: {device}")
    
    history, val_acc = train_crop_model(
        CROP_TO_TRAIN, 
        epochs=EPOCHS, 
        batch_size=BATCH_SIZE, 
        backbone=BACKBONE
    )
else:
    print(f"❌ Unknown crop: {CROP_TO_TRAIN}")
    print(f"Available: {list(CROP_CONFIGS.keys())}")

## 8️⃣ Train All Crops (Full Run)

⚠️ This will take a while! Only run if you want to train models for all crops.

In [ ]:
# ========================================
# 🚀 TRAIN ALL CROPS
# ========================================

EPOCHS = 40
BATCH_SIZE = 16

results_summary = {}

for crop_id in CROP_CONFIGS.keys():
    try:
        print(f"\n\n{'#'*70}")
        print(f"# Training crop {list(CROP_CONFIGS.keys()).index(crop_id)+1}/{len(CROP_CONFIGS)}: {crop_id}")
        print(f"{'#'*70}")
        
        history, val_acc = train_crop_model(crop_id, epochs=EPOCHS, batch_size=BATCH_SIZE)
        
        if val_acc is not None:
            results_summary[crop_id] = {
                'accuracy': float(val_acc),
                'status': 'success'
            }
        else:
            results_summary[crop_id] = {'status': 'skipped', 'error': 'No images found'}
            
    except Exception as e:
        print(f"❌ Failed to train {crop_id}: {e}")
        results_summary[crop_id] = {'status': 'failed', 'error': str(e)}

# Save summary
with open(MODEL_OUTPUT / 'training_summary.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("\n" + "="*70)
print("📊 TRAINING SUMMARY")
print("="*70)
for crop, res in results_summary.items():
    if res['status'] == 'success':
        print(f"  ✅ {crop}: Accuracy={res['accuracy']:.4f}")
    elif res['status'] == 'skipped':
        print(f"  ⏭️ {crop}: SKIPPED - {res.get('error', 'Unknown')}")
    else:
        print(f"  ❌ {crop}: FAILED - {res.get('error', 'Unknown error')}")

## 9️⃣ View Trained Models

After training, your models are saved in:
`E:/FasalVaidya/backend/ml/models_pytorch/<crop_id>/`

Each folder contains:
- `<crop>_best.pth` - Best model weights
- `metadata.json` - Training info and metrics

In [ ]:
# List trained models
print("📦 Trained PyTorch models:")
if MODEL_OUTPUT.exists():
    for crop_dir in MODEL_OUTPUT.iterdir():
        if crop_dir.is_dir():
            files = list(crop_dir.glob('*.pth'))
            if files:
                print(f"  {crop_dir.name}/")
                for f in files:
                    size_mb = f.stat().st_size / (1024*1024)
                    print(f"    - {f.name} ({size_mb:.1f} MB)")
                # Show metadata if exists
                metadata_file = crop_dir / 'metadata.json'
                if metadata_file.exists():
                    with open(metadata_file) as f:
                        meta = json.load(f)
                    print(f"      Accuracy: {meta.get('val_accuracy', 'N/A'):.4f}")
                    print(f"      Device: {meta.get('device', 'N/A')}")
else:
    print("  No models found yet. Run training first!")